# Домашняя работа
### GeekBrains
## Казанцев Виталий

### Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

*   Наименование вакансии.
*   Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
*   Ссылку на саму вакансию.
*   Сайт, откуда собрана вакансия.

***По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.**

версия python 3.9.7

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
from pprint import pprint

In [2]:
headers = {'UserAgent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'}

In [3]:
urlhh = 'http://www.hh.ru'
urlsj = 'https://russia.superjob.ru'

In [4]:
vocation = input('Укажита вакансию:\n')
vocation

Укажита вакансию:
Python


'Python'

https://hh.ru/search/vacancy?area=113&fromSearchLine=true&text=Python

https://russia.superjob.ru/vacancy/search/?keywords=Python

In [5]:
paramshh = {
    'area': '113',
    'fromSearchLine': 'true',
    'text': vocation
}
paramshh

paramssj = {
    'keywords': vocation,
}

In [6]:
response_hh = requests.get(
    urlhh + '/search/vacancy',
    params=paramshh, 
    headers=headers
)
response_hh

<Response [404]>

In [7]:
response_hh = requests.get(
    'https://hh.ru/search/vacancy?area=113&fromSearchLine=true&text=Python', 
    params=None, 
    headers=headers
)
response_hh

<Response [404]>

In [8]:
response_sj = requests.get(
    f'{urlsj}/vacancy/search/', 
    params=paramssj, 
    headers= headers
)
response_sj

<Response [200]>

In [9]:
dom_js = BeautifulSoup(response_sj.text, 'html.parser')

In [10]:
superjobs = dom_js.find_all('div', {'class': 'f-test-search-result-item'})
superjobs[0]

<div class="f-test-search-result-item"><div><div><div class="_3eZwq iJCa5 f-test-vacancy-item _1fma_ _2nteL"><div class="Fo44F QiY08 LvoDO" spacing="3"><div class="_2g1F-"></div><div class="_2g1F-"></div><div class="_2g1F-"><div><div class="jNMYr GPKTZ _1tH7S"><span class="_1e6dO _1XzYb _2EZcW"><a class="icMQ_ _6AfZ9 f-test-link-QA-avtomatizator_(python) _2JivQ _1UJAN" href="/vakansii/qa-avtomatizator-39705235.html" target="_blank">QA-автоматизатор (<span class="_1rS-s">python</span>)</a></span><span class="_1OuF_ _1qw9T f-test-text-company-item-salary"><span class="_2Wp8I _1e6dO _1XzYb Js9sN _3Jn4o">По договорённости</span></span></div><div class="_3_eyK _3P0J7 _9_FPy" spacing="2"><div class="_2g1F-"><span class="_3Fsn4 f-test-text-vacancy-item-company-name _1AZJ2 _1XzYb Js9sN _3Jn4o WGREZ"><a class="icMQ_ _205Zx f-test-link-Sberbank_Rossii _25-u7" href="/clients/sberbank-rossii-297921.html" target="_self">Сбербанк России</a></span></div><div class="_2g1F-"><span class="f-test-text-co

In [11]:
# <div class="we08m">
def class_exists(soup, teg, cl):
    if soup.find(teg, {'class': cl}):
        return True
    return False

In [12]:
superjobs_list = []
for i in range(8):
    paramssj['page'] = i+1
    for job in superjobs:
        if class_exists(job, 'div', 'Fo44F'):
            superjobs_data = {}
            info = job.find('a', {'class': 'icMQ_'})
            name = info.getText()
            link = urlsj + info['href']
            salary = job.find('span', {'class': '_2Wp8I'}).getText()
            salary = salary.replace(u'\xa0', u' ')
            if salary == 'По договорённости':
                mins = None
                maxs = None
                valuta = None
            else:
                salary_list = salary.split(' ')
                valuta = salary_list[-1]
                salary_list.pop()
                if salary_list[0].isalpha():
                    salary_list.pop(0)
                salary = ''.join(salary_list)
                if '—' in salary:
                    mins = int(salary.split('—')[0])
                    maxs = int(salary.split('—')[1])
                else:
                    mins = int(salary)
                    maxs = int(salary)
            site = urlsj
            superjobs_data['name'] = name
            superjobs_data['link'] = link
            superjobs_data['min_salary'] = mins
            superjobs_data['max_salary'] = maxs
            superjobs_data['valut_salary'] = valuta
            superjobs_data['site_from'] = site
            superjobs_list.append(superjobs_data)
        
pprint(superjobs_list[:10])            

[{'link': 'https://russia.superjob.ru/vakansii/qa-avtomatizator-39705235.html',
  'max_salary': None,
  'min_salary': None,
  'name': 'QA-автоматизатор (python)',
  'site_from': 'https://russia.superjob.ru',
  'valut_salary': None},
 {'link': 'https://russia.superjob.ru/vakansii/prepodavatel-programmirovaniya-na-yazyke-python-39552087.html',
  'max_salary': None,
  'min_salary': None,
  'name': 'Преподаватель программирования на языке Python, преподаватель '
          'кибергигиены, мобильной разработки',
  'site_from': 'https://russia.superjob.ru',
  'valut_salary': None},
 {'link': 'https://russia.superjob.ru/vakansii/python-backend-developer-39595398.html',
  'max_salary': 185000,
  'min_salary': 185000,
  'name': 'Python Backend Developer / WEB-разработчик',
  'site_from': 'https://russia.superjob.ru',
  'valut_salary': 'руб.'},
 {'link': 'https://russia.superjob.ru/vakansii/programmist-python-39759582.html',
  'max_salary': None,
  'min_salary': None,
  'name': 'Программист Python

In [13]:
df = pd.DataFrame(superjobs_list)
df.head()

,name,link,min_salary,max_salary,valut_salary,site_from
0,QA-автоматизатор (python),https://russia.superjob.ru/vakansii/qa-avtomat...,NaN,NaN,None,https://russia.superjob.ru
1,Преподаватель программирования на языке Python...,https://russia.superjob.ru/vakansii/prepodavat...,NaN,NaN,None,https://russia.superjob.ru
2,Python Backend Developer / WEB-разработчик,https://russia.superjob.ru/vakansii/python-bac...,185000.0,185000.0,руб.,https://russia.superjob.ru
3,Программист Python,https://russia.superjob.ru/vakansii/programmis...,NaN,NaN,None,https://russia.superjob.ru
4,Senior python developer / Teamlead (Odoo),https://russia.superjob.ru/vakansii/senior-pyt...,180000.0,180000.0,руб.,https://russia.superjob.ru


In [14]:
df.to_csv('superjob.csv', sep='\t')